In [54]:
from transformers import pipeline
import pandas as pd
import time

## NVDIA HIVE Model

### Google

In [8]:
google = pd.read_csv("google_top_search_ppl.csv")

In [9]:
google.head()

,ppl,position,thumbnail,source,source_logo,title,link,original,sightengine
0,Donald Trump,1,https://encrypted-tbn0.gstatic.com/images?q=tb...,"Wikipedia, the free encyclopedia",https://encrypted-tbn1.gstatic.com/faviconV2?u...,Donald Trump - Wikipedia,https://en.wikipedia.org/wiki/Donald_Trump,https://upload.wikimedia.org/wikipedia/commons...,0.01
1,Donald Trump,2,https://encrypted-tbn0.gstatic.com/images?q=tb...,"Wikipedia, the free encyclopedia",https://encrypted-tbn1.gstatic.com/faviconV2?u...,Donald Trump - Wikipedia,https://en.wikipedia.org/wiki/Donald_Trump,https://upload.wikimedia.org/wikipedia/commons...,0.01
2,Donald Trump,3,https://encrypted-tbn0.gstatic.com/images?q=tb...,The White House,https://encrypted-tbn1.gstatic.com/faviconV2?u...,President Donald J. Trump,https://www.whitehouse.gov/administration/dona...,https://www.whitehouse.gov/wp-content/uploads/...,0.01
3,Donald Trump,4,https://encrypted-tbn0.gstatic.com/images?q=tb...,History.com,https://encrypted-tbn1.gstatic.com/faviconV2?u...,"Donald Trump: Presidency, Family, Education | ...",https://www.history.com/articles/donald-trump,https://res.cloudinary.com/aenetworks/image/up...,0.02
4,Donald Trump,5,https://encrypted-tbn0.gstatic.com/images?q=tb...,Biography,https://encrypted-tbn1.gstatic.com/faviconV2?u...,"Donald Trump: Biography, U.S. President, Busin...",https://www.biography.com/political-figures/do...,https://hips.hearstapps.com/hmg-prod/images/ge...,0.01


In [55]:
from dotenv import load_dotenv
import os
from serpapi import GoogleSearch
load_dotenv() 
header_auth_key = os.getenv("NVDIA_API_KEY")

In [56]:
import requests
import base64
from io import BytesIO

header_auth = f"Bearer {header_auth_key}"
invoke_url = "https://ai.api.nvidia.com/v1/cv/hive/deepfake-image-detection"

In [ ]:
# Download the image into memory
response = requests.get(image_url)
image_bytes = response.content
image_b64 = base64.b64encode(image_bytes).decode()

# Helper to upload asset (for large images)
def upload_asset_from_bytes(image_bytes, desc):
    assets_url = "https://api.nvcf.nvidia.com/v2/nvcf/assets"
    headers = {
        "Content-Type": "application/json",
        "Authorization": header_auth,
        "accept": "application/json",
    }
    payload = {
        "contentType": "image/png",
        "description": desc
    }
    response = requests.post(assets_url, headers=headers, json=payload, timeout=30)
    asset_id = response.json()["assetId"]
    upload_url = response.json()["uploadUrl"]

    upload_headers = {
        "Content-Type": "image/png",
        "x-amz-meta-nvcf-asset-description": desc,
    }

    response = requests.put(upload_url, data=image_bytes, headers=upload_headers, timeout=300)
    return asset_id

# If image is small enough, use base64
if len(image_b64) < 180_000:
    payload = {
        "input": [f"data:image/png;base64,{image_b64}"]
    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": header_auth,
        "Accept": "application/json",
    }
else:
    # Upload image as asset
    asset_id = upload_asset_from_bytes(image_bytes, "Image from URL")
    payload = {
        "input": [f"data:image/png;asset_id,{asset_id}"]
    }
    headers = {
        "Content-Type": "application/json",
        "NVCF-INPUT-ASSET-REFERENCES": asset_id,
        "Authorization": header_auth,
    }

# Send request to NVIDIA API
response = requests.post(invoke_url, headers=headers, json=payload)
content = response.json()



In [57]:
image_urls = google['thumbnail'].tolist()

In [58]:
deepfake_scores = []

for url in image_urls:
    try:
        time.sleep(5) 
        # Download image from URL
        img_resp = requests.get(url)
        img_bytes = img_resp.content
        img_b64 = base64.b64encode(img_bytes).decode()

        # Build payload
        payload = {
            "input": [f"data:image/png;base64,{img_b64}"]
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": header_auth,
            "Accept": "application/json"
        }

        # Send request
        resp = requests.post(invoke_url, headers=headers, json=payload)

        if resp.status_code == 200:
            data = resp.json()
            score = data["data"][0]["bounding_boxes"][0]["is_deepfake"]
            deepfake_scores.append(score)
        else:
            print(f"❌ {url} → Error {resp.status_code}")
            deepfake_scores.append(None)

    except Exception as e:
        print(f"⚠️ Failed to process {url}: {e}")
        deepfake_scores.append(None)

⚠️ Failed to process https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRowVpUEitNAnBrZBCXlYK7e0UNiarMRmXOnA&s: list index out of range
⚠️ Failed to process https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ966z8_von1xUFUX_uUllnJ4b-Yklk4NrUog&s: list index out of range
⚠️ Failed to process https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFMbl41EtNC7PNKRzGFq_V3vS4PAcNvhMLLg&s: list index out of range
⚠️ Failed to process https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR3Et5LG-tPx4tm8IkmNWfHQ5JYVeg22yaSuA&s: list index out of range
⚠️ Failed to process https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRbaZu0Rd6Vehx-8VjbOzqjAKt7m2gLCqJw9g&s: list index out of range
⚠️ Failed to process https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSmfsQ5nk0TivPZIEowMcqbxPkLoGJW5h9YRA&s: list index out of range
⚠️ Failed to process https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS7hB9iCa2yvuNfMJlFVGnlh0chRVGdK0ga6g&s: list index out of range
⚠️ Failed to process https:

In [60]:
google['deepfake_score'] = deepfake_scores

In [62]:
google.to_csv("google_top_search_ppl.csv", index=False)

In [77]:
google['sightengine'] = pd.to_numeric(google['sightengine'], errors='coerce')
google.sort_values(by='sightengine', ascending=False).head(5)

,ppl,position,thumbnail,source,source_logo,title,link,original,sightengine,deepfake_score
751,Jill Stein,52,https://encrypted-tbn0.gstatic.com/images?q=tb...,Chicago Tribune,https://encrypted-tbn3.gstatic.com/faviconV2?u...,Jill Stein's HS classmates criticize here pres...,https://www.chicagotribune.com/2024/11/02/jill...,https://www.chicagotribune.com/wp-content/uplo...,0.08,7.026106e-03
653,Mike Tyson,54,https://encrypted-tbn0.gstatic.com/images?q=tb...,Mike Tyson Collection · In stock,https://encrypted-tbn1.gstatic.com/faviconV2?u...,Tyson Pro Competition Gloves - 8oz,https://miketyson.com/products/tyson-pro-compe...,https://miketyson.com/cdn/shop/files/tyson-pro...,0.05,1.818071e-08
413,"Catherine, Princess of Wales",14,https://encrypted-tbn0.gstatic.com/images?q=tb...,Fandom,https://encrypted-tbn2.gstatic.com/faviconV2?u...,"Catherine, Princess of Wales | Monarchies Wiki...","https://monarchies.fandom.com/wiki/Catherine,_...",https://static.wikia.nocookie.net/monarchy-of-...,0.05,1.162635e-07
57,Donald Trump,58,https://encrypted-tbn0.gstatic.com/images?q=tb...,Rolling Stone,https://encrypted-tbn2.gstatic.com/faviconV2?u...,Donald Trump's Imperialist Threat to Take Over...,https://www.rollingstone.com/politics/politica...,https://www.rollingstone.com/wp-content/upload...,0.04,NaN
835,Usher,36,https://encrypted-tbn0.gstatic.com/images?q=tb...,ABC7,https://encrypted-tbn1.gstatic.com/faviconV2?u...,Usher advocates for diabetes awareness at Capi...,https://abc7.com/post/usher-advocates-diabetes...,https://cdn.abcotvs.com/dip/images/14944436_06...,0.04,4.278531e-06


In [78]:
google['deefake_score'] = pd.to_numeric(google['deepfake_score'], errors='coerce')
google.sort_values(by='deepfake_score', ascending=False).head(5)

,ppl,position,thumbnail,source,source_logo,title,link,original,sightengine,deepfake_score,deefake_score
31,Donald Trump,32,https://encrypted-tbn0.gstatic.com/images?q=tb...,City Journal,https://encrypted-tbn0.gstatic.com/faviconV2?u...,"Actually, Donald Trump Is a Republican",https://www.city-journal.org/article/donald-tr...,https://media4.manhattan-institute.org/wp-cont...,0.01,0.911818,0.911818
903,Imane Khelif,4,https://encrypted-tbn0.gstatic.com/images?q=tb...,Hindustan Times,https://encrypted-tbn1.gstatic.com/faviconV2?u...,Imane Khelif speaks out after leaked report cl...,https://www.hindustantimes.com/world-news/us-n...,https://www.hindustantimes.com/ht-img/img/2024...,0.01,0.567429,0.567429
251,JD Vance,52,https://encrypted-tbn0.gstatic.com/images?q=tb...,Ideastream,https://encrypted-tbn0.gstatic.com/faviconV2?u...,Vice president-elect JD Vance resigns from the...,https://www.ideastream.org/show/sound-of-ideas...,https://npr.brightspotcdn.com/dims4/default/a1...,0.01,0.465009,0.465009
520,Tim Walz,21,https://encrypted-tbn0.gstatic.com/images?q=tb...,Governing Magazine,https://encrypted-tbn2.gstatic.com/faviconV2?u...,What Tim Walz Is Known for in Minnesota,https://www.governing.com/politics/what-tim-wa...,https://erepublic.brightspotcdn.com/dims4/defa...,0.01,0.162861,0.162861
981,Imane Khelif,82,https://encrypted-tbn0.gstatic.com/images?q=tb...,Hindustan Times,https://encrypted-tbn1.gstatic.com/faviconV2?u...,Imane Khelif stuns internet with her 'absolute...,https://www.hindustantimes.com/world-news/us-n...,https://images.hindustantimes.com/img/2024/08/...,0.01,0.130441,0.130441


In [79]:
google.sort_values(by='deepfake_score', ascending=False).head(5)['thumbnail'].tolist()

['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQuK2SL3x7y-MX-pLvAK_aUphN9NGflYiJDxQ&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQpoUQ1pLfk5GEX7sbieQYxp0A3CPhT7NSSKw&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRNxmc8so2nIJ6zLhKIhZLP-_snU8QGvphRZg&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR99bSgatx9vMwTOsA-SbaOs4wra6y4qcLseg&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS2SCNs4f3lp4UHMiUhPslLqMzoqpt4AQN9PA&s']

In [130]:
google['deepfake_score'].isna().sum()

73

### DuckDuckGo

In [64]:
duckduckgo = pd.read_csv("top_search_people_images.csv")
duckduckgo.head()

,Person,Image_URL,sightengine
0,Donald Trump,https://i.ytimg.com/vi/ouelYIRr220/maxres2.jpg...,0.02
1,Donald Trump,https://i.ytimg.com/vi/xBacsGFXKB0/maxres2.jpg...,0.01
2,Donald Trump,https://www.sinembargo.mx/wp-content/uploads/2...,0.01
3,Donald Trump,https://i.ytimg.com/vi/DrpcIXe2V0w/maxresdefau...,0.01
4,Donald Trump,https://i.ytimg.com/vi/RdG4dVRj2gg/maxres2.jpg...,0.02


In [65]:
image_urls = duckduckgo['Image_URL'].tolist()
deepfake_scores = []
for url in image_urls:
    try:
        time.sleep(5) 
        # Download image from URL
        img_resp = requests.get(url)
        img_bytes = img_resp.content
        img_b64 = base64.b64encode(img_bytes).decode()

        # Build payload
        payload = {
            "input": [f"data:image/png;base64,{img_b64}"]
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": header_auth,
            "Accept": "application/json"
        }

        # Send request
        resp = requests.post(invoke_url, headers=headers, json=payload)

        if resp.status_code == 200:
            data = resp.json()
            score = data["data"][0]["bounding_boxes"][0]["is_deepfake"]
            deepfake_scores.append(score)
        else:
            print(f"❌ {url} → Error {resp.status_code}")
            deepfake_scores.append(None)

    except Exception as e:
        print(f"⚠️ Failed to process {url}: {e}")
        deepfake_scores.append(None)

❌ https://www.sinembargo.mx/wp-content/uploads/2025/06/el-presidente-donald-trump.jpg?x63043 → Error 500
⚠️ Failed to process https://i.ytimg.com/vi/Ytsnss5oJns/maxresdefault.jpg?sqp=-oaymwEoCIAKENAF8quKqQMcGADwAQH4AbYIgAKAD4oCDAgAEAEYciBbKCQwDw==&rs=AOn4CLAK5oL0hHVI-opZAo7LX42g1_Lw6g: list index out of range
⚠️ Failed to process https://i.ytimg.com/vi/TdOuoxAGOaY/maxresdefault.jpg: list index out of range
⚠️ Failed to process https://i.ytimg.com/vi/6DiyBoh_hf8/maxres2.jpg?sqp=-oaymwEoCIAKENAF8quKqQMcGADwAQH4Ac4FgAKACooCDAgAEAEYZSBYKEYwDw==&rs=AOn4CLAeS3kyKNXCBXv5XDDPCoH1gXoxkg: list index out of range
⚠️ Failed to process https://i.ytimg.com/vi/__6DPExaVKQ/maxresdefault.jpg?sqp=-oaymwEmCIAKENAF8quKqQMa8AEB-AG0CIACgA-KAgwIABABGEYgRyhZMA8=&rs=AOn4CLDkqYhnaSVTZePw39_MaF3V4l87OQ: list index out of range
⚠️ Failed to process https://i.ytimg.com/vi/X6Jkefj5Yxo/maxresdefault.jpg?sqp=-oaymwEmCIAKENAF8quKqQMa8AEB-AHeA4AC1gaKAgwIABABGH8gLygoMA8=&rs=AOn4CLBxYjaS2TQEodsVRjGtyQj8XnW4AA: list index

In [66]:
duckduckgo['deepfake_score'] = deepfake_scores

In [68]:
duckduckgo.to_csv("top_search_people_images.csv", index=False)

sightengine found 5 images with deepfake scores above 0.5 in a total of 206 valid searches in ddg

hivemodel also found 5 images with deefake scores above 0.5 in a total of 484 valid searches in ddg

In [104]:
# sightengine processing
duckduckgo['sightengine'] = pd.to_numeric(duckduckgo['sightengine'], errors='coerce')
duckduckgo[duckduckgo['sightengine']> 0.5].sort_values(by='sightengine', ascending=False)

,Person,Image_URL,sightengine,deepfake_score,deefake_score
6,Donald Trump,https://i.ytimg.com/vi/Ytsnss5oJns/maxresdefau...,0.99,NaN,NaN
32,Donald Trump,https://i.ytimg.com/vi/PExqrkMVmek/maxres2.jpg...,0.99,1.035205e-05,1.035205e-05
42,Donald Trump,https://i.ytimg.com/vi/q6OAzlc-9VM/maxres2.jpg...,0.99,4.240402e-08,4.240402e-08
236,Joe Biden,https://cnnespanol.cnn.com/wp-content/uploads/...,0.90,1.095145e-08,1.095145e-08
210,Joe Biden,https://img-s-msn-com.akamaized.net/tenant/amp...,0.72,NaN,NaN


In [132]:
duckduckgo[duckduckgo['sightengine'].notna()].shape

(206, 5)

In [131]:
duckduckgo[duckduckgo['deefake_score'].notna()].shape

(484, 5)

In [81]:
duckduckgo['deefake_score'] = pd.to_numeric(duckduckgo['deepfake_score'], errors='coerce')
duckduckgo.sort_values(by='deepfake_score', ascending=False).head(5)

,Person,Image_URL,sightengine,deepfake_score,deefake_score
34,Donald Trump,https://i.ytimg.com/vi/dHlsPkwJ2-U/maxres2.jpg...,0.01,0.999653,0.999653
652,Imane Khelif,https://www.totalprosports.com/wp-content/uplo...,NaN,0.981303,0.981303
671,Imane Khelif,https://hype.my/wp-content/uploads/2024/08/ima...,NaN,0.921635,0.921635
623,Usher,https://i.ytimg.com/vi/CM55atQxemg/maxresdefau...,NaN,0.657769,0.657769
433,Mike Tyson,https://i.ytimg.com/vi/vvYD1ilotH0/maxres2.jpg...,NaN,0.468463,0.468463


In [84]:
duckduckgo.sort_values(by='deepfake_score', ascending=False).head(5)['Image_URL'].tolist()

['https://i.ytimg.com/vi/dHlsPkwJ2-U/maxres2.jpg?sqp=-oaymwEoCIAKENAF8quKqQMcGADwAQH4Ac4FgAKACooCDAgAEAEYZSBeKFAwDw==&rs=AOn4CLBgBx94zcZlkPvzIzsFrFU2Pyshqg',
 'https://www.totalprosports.com/wp-content/uploads/2024/08/Imane-Khelif-1-enh.jpg',
 'https://hype.my/wp-content/uploads/2024/08/imane-khelif4.jpg',
 'https://i.ytimg.com/vi/CM55atQxemg/maxresdefault.jpg',
 'https://i.ytimg.com/vi/vvYD1ilotH0/maxres2.jpg?sqp=-oaymwEoCIAKENAF8quKqQMcGADwAQH4AbYIgAKgC4oCDAgAEAEYZCBKKGUwDw==&rs=AOn4CLCQ5PqoTsORvNHgdmZ21J3VIL3gVw']

### Bing Image

In [70]:
bing = pd.read_csv("bing_top_search_ppl.csv")
bing.head()

,ppl,position,thumbnail,source,source_logo,title,link,original,sightengine
0,Donald Trump,1,https://tse3.mm.bing.net/th/id/OIP.0FYH3xpZSlO...,https://www.elmundo.es/e/do/donald-trump.html,https://tse1.mm.bing.net/th/id/ODF.EkAJUUdXlpp...,Donald Trump - Noticias de Donald Trump | EL M...,https://www.bing.com/images/search?view=detail...,https://e00-elmundo.uecdn.es/assets/multimedia...,0.01
1,Donald Trump,2,https://tse4.mm.bing.net/th/id/OIP.5TjtSLeKD_V...,https://www.zdf.de/nachrichten/politik/ausland...,NaN,Prozess gegen Trump: Amerikas wichtigstes Cast...,https://www.bing.com/images/search?view=detail...,https://www.zdf.de/assets/donald-trump-prozess...,0.01
2,Donald Trump,3,https://tse2.mm.bing.net/th/id/OIP.j5HHw92jQoZ...,https://www.usatoday.com/story/news/politics/2...,NaN,Trump was arrested today. Here are the key tak...,https://www.bing.com/images/search?view=detail...,https://www.usatoday.com/gcdn/presto/2023/04/0...,0.01
3,Donald Trump,4,https://tse4.mm.bing.net/th/id/OIP.9gQs6csqXYL...,https://puntoconvergente.uca.edu.ar/quien-es-d...,NaN,"Donald Trump, el ""outsider"" que rompió los man...",https://www.bing.com/images/search?view=detail...,https://puntoconvergente.uca.edu.ar/wp-content...,0.01
4,Donald Trump,5,https://tse2.mm.bing.net/th/id/OIP.L78k6WG8II3...,https://www.foxnews.com/opinion/president-trum...,https://tse4.mm.bing.net/th/id/ODF.mdH-v7wPV5p...,President Trump's closing argument: Vote Repub...,https://www.bing.com/images/search?view=detail...,https://static.foxnews.com/foxnews.com/content...,0.01


In [71]:
image_urls = bing['thumbnail'].tolist()
deepfake_scores = []
for url in image_urls:
    try:
        time.sleep(5) 
        # Download image from URL
        img_resp = requests.get(url)
        img_bytes = img_resp.content
        img_b64 = base64.b64encode(img_bytes).decode()

        # Build payload
        payload = {
            "input": [f"data:image/png;base64,{img_b64}"]
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": header_auth,
            "Accept": "application/json"
        }

        # Send request
        resp = requests.post(invoke_url, headers=headers, json=payload)

        if resp.status_code == 200:
            data = resp.json()
            score = data["data"][0]["bounding_boxes"][0]["is_deepfake"]
            deepfake_scores.append(score)
        else:
            print(f"❌ {url} → Error {resp.status_code}")
            deepfake_scores.append(None)

    except Exception as e:
        print(f"⚠️ Failed to process {url}: {e}")
        deepfake_scores.append(None)

⚠️ Failed to process https://tse2.mm.bing.net/th/id/OIP.RvLmUzsj9a7OFt7dhFpE2QHaEK?w=280&h=180&c=7&r=0&o=7&pid=1.7&rm=3: list index out of range
⚠️ Failed to process https://tse4.mm.bing.net/th/id/OIP.7J4e0ZSP-XjYV3TcCnLbKgHaEK?w=269&h=180&c=7&r=0&o=7&pid=1.7&rm=3: list index out of range
⚠️ Failed to process https://tse3.mm.bing.net/th/id/OIP.Q7FQUsUr-DakQ9VL799jxQHaKC?w=115&h=180&c=7&r=0&o=7&pid=1.7&rm=3: list index out of range
⚠️ Failed to process https://tse2.mm.bing.net/th/id/OIP.-cuZJ9f4Z7UUzwc6q3FAZQHaEz?w=258&h=180&c=7&r=0&o=7&pid=1.7&rm=3: list index out of range
⚠️ Failed to process https://tse2.mm.bing.net/th/id/OIP.xUAgK0aZ5uqvReQ_sdzErAHaEK?w=288&h=180&c=7&r=0&o=7&pid=1.7&rm=3: list index out of range
⚠️ Failed to process https://ts2.mm.bing.net/th?id=OIP.z-N-1Gsyj8HFJAfmQeNy4gHaJQ&pid=15.1: list index out of range
⚠️ Failed to process https://tse1.mm.bing.net/th/id/OIP.FLp7BoWLCFZifglaYuyM8wHaId?w=150&h=180&c=7&r=0&o=7&pid=1.7&rm=3: list index out of range
⚠️ Failed to p

In [72]:
bing['deepfake_score'] = deepfake_scores
bing.to_csv("bing_top_search_ppl.csv", index=False)

sightengine found 3 out of 545 by hive, and 2 out of 608. 

In [112]:
bing[bing['deepfake_score'].notna()].shape

(545, 11)

In [113]:
bing[bing['sightengine'].notna()].shape

(608, 11)

In [111]:
bing['sightengine'] = pd.to_numeric(bing['sightengine'], errors='coerce')
bing[bing['sightengine'] > 0.5].sort_values(by='sightengine', ascending=False)

,ppl,position,thumbnail,source,source_logo,title,link,original,sightengine,deepfake_score,deefake_score
452,Mike Tyson,20,https://ts2.explicit.bing.net/th?id=OIP.eQmvmF...,NaN,NaN,NaN,https://www.bing.com/images/search?view=detail...,https://tattoogunmachine.com/wp-content/upload...,0.99,0.000002,0.000002
508,Jill Stein,11,https://tse3.mm.bing.net/th/id/OIP._X5tf5rs9M8...,https://www.deviantart.com/suitan760/art/Jill-...,NaN,Jill Valentine - RE3 Remake by Suitan760 on De...,https://www.bing.com/images/search?view=detail...,https://images-wixmp-ed30a86b8c4ca887773594c2....,0.99,0.000240,0.000240


In [133]:
bing[bing['sightengine'] > 0.5]['thumbnail'].tolist()

['https://ts2.explicit.bing.net/th?id=OIP.eQmvmF7Bnu_exh1WWXNE1wHaHa&pid=15.1',
 'https://tse3.mm.bing.net/th/id/OIP._X5tf5rs9M8R9o0-6NOXxAHaLZ?w=194&h=299&c=7&r=0&o=7&pid=1.7&rm=3']

In [109]:
bing['deefake_score'] = pd.to_numeric(bing['deepfake_score'], errors='coerce')
bing[bing['deefake_score'] > 0.5].sort_values(by='deefake_score', ascending=False)


,ppl,position,thumbnail,source,source_logo,title,link,original,sightengine,deepfake_score,deefake_score
525,Usher,17,https://ts2.mm.bing.net/th?id=OIP.lhZv-DOmPwe8...,NaN,NaN,NaN,https://www.bing.com/images/search?view=detail...,https://hairstylecamp.com/wp-content/uploads/u...,0.01,0.742729,0.742729
64,Kamala Harris,40,https://ts4.mm.bing.net/th?id=OIP.19A6pStLMevj...,NaN,NaN,NaN,https://www.bing.com/images/search?view=detail...,https://cdn.britannica.com/70/221770-050-30348...,0.01,0.692952,0.692952
320,"Catherine, Princess of Wales",9,https://tse2.mm.bing.net/th?id=OIF.PG%2buGMegF...,https://www.youtube.com/watch?v=LLktLhimJbQ,NaN,Catherine Princess of Wales - YouTube,https://www.bing.com/images/search?view=detail...,https://i.ytimg.com/vi/LLktLhimJbQ/maxresdefau...,0.07,0.684154,0.684154


In [134]:
bing.sort_values(by='deepfake_score', ascending=False).head(3)['thumbnail'].tolist()

['https://ts2.mm.bing.net/th?id=OIP.lhZv-DOmPwe8aNphS8T9wwAAAA&pid=15.1',
 'https://ts4.mm.bing.net/th?id=OIP.19A6pStLMevjPkVZAWk8OQHaE8&pid=15.1',
 'https://tse2.mm.bing.net/th?id=OIF.PG%2buGMegFriXoxBjwPs%2fvw&w=255&h=180&c=7&r=0&o=7&pid=1.7&rm=3']

### Youtube

In [127]:
YouTube = pd.read_csv("youtube_top_search_ppl.csv")
YouTube = YouTube.drop(columns=['deepfake_score'])
YouTube.head()

,ppl,thumbnail
0,Donald Trump,https://i.ytimg.com/vi/qGleCS-RdIw/hq720.jpg?s...
1,Donald Trump,https://i.ytimg.com/vi/SAnpkcDUGwQ/hq720.jpg?s...
2,Donald Trump,https://i.ytimg.com/vi/72k1s18cnnM/hq720.jpg?s...
3,Donald Trump,https://i.ytimg.com/vi/gaCfndCeQY8/hq720.jpg?s...
4,Donald Trump,https://i.ytimg.com/vi/A-BhAmDLKvc/hq720.jpg?s...


In [129]:
image_urls = YouTube['thumbnail'].tolist()
deepfake_scores = []
for url in image_urls:
    try:
        time.sleep(5) 
        # Download image from URL
        img_resp = requests.get(url)
        img_bytes = img_resp.content
        img_b64 = base64.b64encode(img_bytes).decode()

        # Build payload
        payload = {
            "input": [f"data:image/png;base64,{img_b64}"]
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": header_auth,
            "Accept": "application/json"
        }

        # Send request
        resp = requests.post(invoke_url, headers=headers, json=payload)

        if resp.status_code == 200:
            data = resp.json()
            score = data["data"][0]["bounding_boxes"][0]["is_deepfake"]
            deepfake_scores.append(score)
        else:
            print(f"❌ {url} → Error {resp.status_code}")
            deepfake_scores.append(None)

    except Exception as e:
        print(f"⚠️ Failed to process {url}: {e}")
        deepfake_scores.append(None)

⚠️ Failed to process https://i.ytimg.com/vi/qGleCS-RdIw/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLAb9z73E8SUqYtuJpD5QUBoc38E7Q: list index out of range
❌ https://i.ytimg.com/vi/gaCfndCeQY8/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLChC-KV2zZjc6U0po7eO-R_ZAA-yw → Error 500
❌ https://i.ytimg.com/vi/A-BhAmDLKvc/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLAB1t68nahsTOAL2pkBztVXLPNomw → Error 500
❌ https://i.ytimg.com/vi/9swod2QVU9k/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLDV7CpRoJgtKRVUf4TMhQjaOYMNEQ → Error 500
❌ https://i.ytimg.com/vi/1IQxQyj46lg/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLBOTXesM_mqx2gcQJJr0mPKlA0_Dg → Error 500
❌ https://i.ytimg.com/vi/qsLiUfIswYo/hq720.jpg?sqp=-oaymwFBCNAFEJQDSFryq4qpAzMIARUAAIhCGADYAQHiAQoIGBACGAY4AUAB8AEB-AHOBYACgAqKAgwIABABGGUgTShAMA8=&rs=AOn4CLDR8NniHSCJAuOU4HRRw9J

KeyboardInterrupt: 

In [97]:
YouTube['deepfake_score'] = deepfake_scores
YouTube.to_csv("youtube_top_search_ppl.csv", index=False)

In [98]:
YouTube['deefake_score'] = pd.to_numeric(YouTube['deepfake_score'], errors='coerce')
YouTube.sort_values(by='deepfake_score', ascending=False).head(5)

,ppl,thumbnail,deepfake_score,deefake_score
398,Tim Walz,https://i.ytimg.com/vi/cTF_C_XTxc8/hq720.jpg?s...,5.988959e-06,5.988959e-06
83,Kamala Harris,https://i.ytimg.com/vi/rxCnf1tb5Xs/hq720.jpg?s...,5.320123e-06,5.320123e-06
1,Donald Trump,https://i.ytimg.com/vi/SAnpkcDUGwQ/hq720.jpg?s...,5.220785e-07,5.220785e-07
2,Donald Trump,https://i.ytimg.com/vi/72k1s18cnnM/hq720.jpg?s...,2.356860e-07,2.356860e-07
397,Tim Walz,https://i.ytimg.com/vi/kQvyHlYkPwY/hq720.jpg?s...,2.703960e-10,2.703960e-10


In [99]:
YouTube.sort_values(by='deepfake_score', ascending=False).head(5)['thumbnail'].tolist()

['https://i.ytimg.com/vi/cTF_C_XTxc8/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLCqIYZiApQEJ3JCQOwsBVu-YiMYtw',
 'https://i.ytimg.com/vi/rxCnf1tb5Xs/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLB-da47iORzmzJLNbtm7UJb9SY2rA',
 'https://i.ytimg.com/vi/SAnpkcDUGwQ/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLDItfeRE19GmWf-aUn1oEusXMFUAA',
 'https://i.ytimg.com/vi/72k1s18cnnM/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLB7FfedxNDEt_eShUq9__9ZBFx7ag',
 'https://i.ytimg.com/vi/kQvyHlYkPwY/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLCt-DT6vCKyQ2TPtnhboiKlYWbiFQ']

In [137]:
YouTube[YouTube['deefake_score'].isna()].count()

KeyError: 'deefake_score'

In [125]:
YouTube[YouTube['deepfake_score'].isna()]['thumbnail'].tolist()

['https://i.ytimg.com/vi/qGleCS-RdIw/hq720.jpg?sqp=-oaymwEXCNAFEJQDSFryq4qpAwkIARUAAIhCGAE=&rs=AOn4CLAb9z73E8SUqYtuJpD5QUBoc38E7Q',
 'https://i.ytimg.com/vi/gaCfndCeQY8/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLChC-KV2zZjc6U0po7eO-R_ZAA-yw',
 'https://i.ytimg.com/vi/A-BhAmDLKvc/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLAB1t68nahsTOAL2pkBztVXLPNomw',
 'https://i.ytimg.com/vi/9swod2QVU9k/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLDV7CpRoJgtKRVUf4TMhQjaOYMNEQ',
 'https://i.ytimg.com/vi/1IQxQyj46lg/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLBOTXesM_mqx2gcQJJr0mPKlA0_Dg',
 'https://i.ytimg.com/vi/qsLiUfIswYo/hq720.jpg?sqp=-oaymwFBCNAFEJQDSFryq4qpAzMIARUAAIhCGADYAQHiAQoIGBACGAY4AUAB8AEB-AHOBYACgAqKAgwIABABGGUgTShAMA8=&rs=AOn4CLDR8NniHSCJAuOU4HRRw9Je1kaWsw',
 'https://i.ytimg.com/vi/Ol7q0gHQedg/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq

In [120]:
YouTube.shape

(794, 4)

### Rumble

In [101]:
rumble = pd.read_csv("rumble_top_ppl_search.csv")
rumble.head()

,Unnamed: 0,ppl,thumbnail
0,0,Donald Trump,https://1a-1791.com/video/fww1/cb/s8/1/J/C/h/X...
1,1,Donald Trump,https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P...
2,2,Donald Trump,https://1a-1791.com/video/fww1/b5/s8/1/J/c/1/R...
3,3,Donald Trump,https://1a-1791.com/video/fww1/e9/s8/1/l/X/Y/L...
4,4,Donald Trump,https://1a-1791.com/video/fww1/8b/s8/1/P/9/F/L...


In [102]:
image_urls = rumble['thumbnail'].tolist()
deepfake_scores = []
for url in image_urls:
    try:
        time.sleep(5) 
        # Download image from URL
        img_resp = requests.get(url)
        img_bytes = img_resp.content
        img_b64 = base64.b64encode(img_bytes).decode()

        # Build payload
        payload = {
            "input": [f"data:image/png;base64,{img_b64}"]
        }
        headers = {
            "Content-Type": "application/json",
            "Authorization": header_auth,
            "Accept": "application/json"
        }

        # Send request
        resp = requests.post(invoke_url, headers=headers, json=payload)

        if resp.status_code == 200:
            data = resp.json()
            score = data["data"][0]["bounding_boxes"][0]["is_deepfake"]
            deepfake_scores.append(score)
        else:
            print(f"❌ {url} → Error {resp.status_code}")
            deepfake_scores.append(None)

    except Exception as e:
        print(f"⚠️ Failed to process {url}: {e}")
        deepfake_scores.append(None)

⚠️ Failed to process https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg: list index out of range
⚠️ Failed to process https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg: list index out of range
⚠️ Failed to process https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg: list index out of range
⚠️ Failed to process https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg: list index out of range
⚠️ Failed to process https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg: list index out of range
⚠️ Failed to process https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg: list index out of range
⚠️ Failed to process https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg: list index out o

In [114]:
rumble['deepfake_score'] = deepfake_scores
rumble.to_csv("rumble_top_ppl_search.csv", index=False)

In [115]:
rumble[rumble['deepfake_score'].notna()].shape

(210, 4)

In [116]:
rumble['deepfake_score'] = pd.to_numeric(rumble['deepfake_score'], errors='coerce')
rumble[rumble['deepfake_score'] > 0.5].sort_values(by='deepfake_score', ascending=False)

,Unnamed: 0,ppl,thumbnail,deepfake_score


In [118]:
rumble[~rumble['deepfake_score'].notna()]['thumbnail'].tolist()

['https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-Biden-Watch-t.jpg',
 'https://1a-1791.com/video/fww1/6a/s8/1/f/E/H/P/fEHPy.oq1b.2-small-6.2.25-NO-Joe-

In [121]:
rumble.shape

(220, 4)